In [4]:
!pip install nltk
import nltk

In [92]:
import itertools

In [ ]:
nltk.download('all-corpora')

## Инструменты для работы в циклах (итераторы)

### zip и zip_longest

In [86]:
for a, b in zip ([1,2,3], [1,2,3]):
  print (a+b)

2
4
6


Также с индексами (переюор массива циклом немножко быстрее, чем обращение по адрессу?)

про то, что могут быть неравными

In [95]:
for t in itertools.zip_longest('ABCD', 'xy', fillvalue='-'):
  print(t)

('A', 'x')
('B', 'y')
('C', '-')
('D', '-')


In [88]:
zipped = zip([1,2,3], [1,2,3])

In [90]:
list(zipped)

[(1, 1), (2, 2), (3, 3)]

### Chain

In [96]:
for t in itertools.chain('ABCD', 'xy'):
  print(t)

A
B
C
D
x
y


In [99]:
for t in itertools.chain.from_iterable(['ABCD', 'xy', 'we']):
  print(t)

A
B
C
D
x
y
w
e


### Всякие фильтры

In [100]:
for t in itertools.compress('ABCDEF', [1,0,1,0,1,1]):
  print(t)

A
C
E
F


In [101]:
for t in itertools.dropwhile(lambda x: x<5, [1,4,6,4,1]):
  print(t)

6
4
1


Вы же знаете, что такое лямбда ?

def lambda ( АРГУМЕНТЫ):

return TRUE ИЛИ FALSE OТ

In [102]:
for t in itertools.takewhile(lambda x: x<5, [1,4,6,4,1]):
  print(t)

1
4


In [105]:
for t in itertools.filterfalse(lambda x: x%2, range(10)):
  print(t)

0
2
4
6
8


Документация https://docs.python.org/3/library/itertools.html

## Синтаксис в NLTK
 #### Грамматика составляющих (помните такую?)


NLTK умеет не только в морфу, но и в синтаксис. Помните, на самом первом занятии, когда мы занимались чанкингом (кстати, это що?), мы быстро начертили какое-то стрёмное дерево и побежали дальше?

In [18]:
sentence = [("the", "DT"), ("little", "JJ"), ("yellow", "JJ"), ("dog", "NN"), ("barked", "VBD"), ("at", "IN"),  ("the", "DT"), ("cat", "NN")]

grammar = "NP: {<DT>?<JJ>*<NN>}"


cp = nltk.RegexpParser(grammar)
result = cp.parse(sentence)
print(result)


(S
  (NP the/DT little/JJ yellow/JJ dog/NN)
  barked/VBD
  at/IN
  (NP the/DT cat/NN))


Так вот сегодня будем разбираться со всем этим подробнее

In [19]:
rules = """
    S -> NP VP
    NP -> Det ADJ N | Det N | NN
    VP -> V NP 
    Det -> 'a'
    ADJ -> 'tasty' | ADV ADJ
    ADV -> 'very'
    N -> 'fish' | 'fork' | 'dog' | 'boy'
    NN -> 'Mary' | 'John'
    V -> 'eats'
""".split('\n')

In [24]:
# сделали парсер
grammar = nltk.CFG.fromstring('\n'.join(rules))
cp = nltk.EarleyChartParser(grammar)

In [29]:
def print_parses(parser, sentence):
    for tree in parser.parse(sentence.split()):
        print(tree)
        
print_parses(cp, "Mary eats a fish")

(S (NP (NN Mary)) (VP (V eats) (NP (Det a) (N fish))))


Давайте посмотрим на иконичное предложение с 2 прочтениями I shot elephant in my pajamas. NLTK умеет выдавать сразу несколько деревьев, если вы ему дадите предложение с возможной неоднозначностью -- главное настроить так грамматику, чтобы он ловил эту неоднозначность и не ломался.

Вывод ниже.

In [26]:


>>> el_grammar = nltk.CFG.fromstring("""

# Your Code here
... S -> NP VP
... PP -> P NP
... NP -> Det N | NP PP | 'I'
... VP -> V NP | VP PP
... Det -> 'an' | 'my'
... N -> 'elephant' | 'pajamas'
... V -> 'shot'
... P -> 'in'
... """)

Заметьте, что парсер поменялся -- там и в NLTK штук 10, но по факту отличаеются они мало.

In [30]:
>>> sent = ['I', 'shot', 'an', 'elephant', 'in', 'my', 'pajamas']
>>> parser = nltk.ChartParser(el_grammar) #
>>> for tree in parser.parse(sent):
...     print(tree)

(S
  (NP I)
  (VP
    (VP (V shot) (NP (Det an) (N elephant)))
    (PP (P in) (NP (Det my) (N pajamas)))))
(S
  (NP I)
  (VP
    (V shot)
    (NP
      (NP (Det an) (N elephant))
      (PP (P in) (NP (Det my) (N pajamas))))))


Вернемся к описанию EarleyChartParser и найдём, как включить отладочный вывод -- чтобы посмотреть на шаги разбора.

In [31]:
cp_verbose = nltk.EarleyChartParser(grammar, trace=1)
print_parses(cp_verbose, "Mary eats a fish")

|.   Mary  .   eats  .    a    .   fish  .|
|[---------]         .         .         .| [0:1] 'Mary'
|.         [---------]         .         .| [1:2] 'eats'
|.         .         [---------]         .| [2:3] 'a'
|.         .         .         [---------]| [3:4] 'fish'
|>         .         .         .         .| [0:0] S  -> * NP VP
|>         .         .         .         .| [0:0] NP -> * Det ADJ N
|>         .         .         .         .| [0:0] NP -> * Det N
|>         .         .         .         .| [0:0] NP -> * NN
|>         .         .         .         .| [0:0] NN -> * 'Mary'
|[---------]         .         .         .| [0:1] NN -> 'Mary' *
|[---------]         .         .         .| [0:1] NP -> NN *
|[--------->         .         .         .| [0:1] S  -> NP * VP
|.         >         .         .         .| [1:1] VP -> * V NP
|.         >         .         .         .| [1:1] V  -> * 'eats'
|.         [---------]         .         .| [1:2] V  -> 'eats' *
|.         [--------->    

In [32]:
# Можно больше отладочного вывода!
very_verbose_cp = nltk.EarleyChartParser(grammar, trace=2)
print_parses(very_verbose_cp, "Mary eats a very tasty fish")

|. Mary . eats .  a   . very .tasty . fish .|
Leaf Init Rule:
|[------]      .      .      .      .      .| [0:1] 'Mary'
|.      [------]      .      .      .      .| [1:2] 'eats'
|.      .      [------]      .      .      .| [2:3] 'a'
|.      .      .      [------]      .      .| [3:4] 'very'
|.      .      .      .      [------]      .| [4:5] 'tasty'
|.      .      .      .      .      [------]| [5:6] 'fish'
Top Down Init Rule:
|>      .      .      .      .      .      .| [0:0] S  -> * NP VP

* Processing queue: 0 

Predictor Rule:
|>      .      .      .      .      .      .| [0:0] NP -> * Det ADJ N
|>      .      .      .      .      .      .| [0:0] NP -> * Det N
|>      .      .      .      .      .      .| [0:0] NP -> * NN
Predictor Rule:
|>      .      .      .      .      .      .| [0:0] NN -> * 'Mary'

* Processing queue: 1 

Scanner Rule:
|[------]      .      .      .      .      .| [0:1] NN -> 'Mary' *
Completer Rule:
|[------]      .      .      .      .      .| [0:1] NP 

If you are interested in experimenting with writing CFGs, you will find it helpful to create and edit your grammar in a text file, say mygrammar.cfg. You can then load it into NLTK and parse with it as follows:

In [37]:
>>> grammar1 = nltk.data.load('file:mygrammar.cfg') #у нас его нет
>>> sent = "Mary saw Bob".split()
>>> rd_parser = nltk.RecursiveDescentParser(grammar1)
>>> for tree in rd_parser.parse(sent):
...      print(tree)

LookupError: ignored

When you write CFGs for parsing in NLTK, you cannot combine grammatical categories with lexical items on the righthand side of the same production. Thus, a production such as PP -> 'of' NP is disallowed. In addition, you are not permitted to place multi-word lexical items on the righthand side of a production. So rather than writing NP -> 'New
York', you have to resort to something like NP -> 'New_York' instead.

## Грамматики зависимостей ( а это что?)
Можно использовать то же предложение, поскольку неоднозначность проявляется и в представлении зависимостей.
В NLTK можно примерно одинаково работать с CFG и зависимостями (`DependencyGrammar`)

In [38]:
dep_rules = """
... 'shot' -> 'I' | 'elephant' | 'in' | 'morning'
... 'morning' -> 'One'
... 'elephant' -> 'an' | 'in'
... 'in' -> 'pajamas'
... 'pajamas' -> 'my'
... """

In [39]:
dep_grammar = nltk.DependencyGrammar.fromstring(dep_rules)

In [40]:
pdp = nltk.ProjectiveDependencyParser(dep_grammar)
sent = 'One morning I shot an elephant in my pajamas'
print_parses(pdp, sent)

(shot (morning One) I (elephant an (in (pajamas my))))
(shot (morning One) I (elephant an) (in (pajamas my)))


## Встроенные грамматики NLTK

In [71]:

>>> from nltk.corpus import treebank
>>> t = treebank.parsed_sents('wsj_0001.mrg')
>>> print(t)


[Tree('S', [Tree('NP-SBJ', [Tree('NP', [Tree('NNP', ['Pierre']), Tree('NNP', ['Vinken'])]), Tree(',', [',']), Tree('ADJP', [Tree('NP', [Tree('CD', ['61']), Tree('NNS', ['years'])]), Tree('JJ', ['old'])]), Tree(',', [','])]), Tree('VP', [Tree('MD', ['will']), Tree('VP', [Tree('VB', ['join']), Tree('NP', [Tree('DT', ['the']), Tree('NN', ['board'])]), Tree('PP-CLR', [Tree('IN', ['as']), Tree('NP', [Tree('DT', ['a']), Tree('JJ', ['nonexecutive']), Tree('NN', ['director'])])]), Tree('NP-TMP', [Tree('NNP', ['Nov.']), Tree('CD', ['29'])])])]), Tree('.', ['.'])]), Tree('S', [Tree('NP-SBJ', [Tree('NNP', ['Mr.']), Tree('NNP', ['Vinken'])]), Tree('VP', [Tree('VBZ', ['is']), Tree('NP-PRD', [Tree('NP', [Tree('NN', ['chairman'])]), Tree('PP', [Tree('IN', ['of']), Tree('NP', [Tree('NP', [Tree('NNP', ['Elsevier']), Tree('NNP', ['N.V.'])]), Tree(',', [',']), Tree('NP', [Tree('DT', ['the']), Tree('NNP', ['Dutch']), Tree('VBG', ['publishing']), Tree('NN', ['group'])])])])])]), Tree('.', ['.'])])]


Посмотрите на эту программу и скажите, что она делает?

In [72]:
def filter(tree):
    child_nodes = [child.label() for child in tree
                   if isinstance(child, nltk.Tree)]
    return  (tree.label() == 'VP') and ('S' in child_nodes)

In [73]:
for tr in t:
  print(tr)
  print(filter(tr))

(S
  (NP-SBJ
    (NP (NNP Pierre) (NNP Vinken))
    (, ,)
    (ADJP (NP (CD 61) (NNS years)) (JJ old))
    (, ,))
  (VP
    (MD will)
    (VP
      (VB join)
      (NP (DT the) (NN board))
      (PP-CLR (IN as) (NP (DT a) (JJ nonexecutive) (NN director)))
      (NP-TMP (NNP Nov.) (CD 29))))
  (. .))
False
(S
  (NP-SBJ (NNP Mr.) (NNP Vinken))
  (VP
    (VBZ is)
    (NP-PRD
      (NP (NN chairman))
      (PP
        (IN of)
        (NP
          (NP (NNP Elsevier) (NNP N.V.))
          (, ,)
          (NP (DT the) (NNP Dutch) (VBG publishing) (NN group))))))
  (. .))
False


Окей, но что нам делать с неоднозначностью?

Когда размер грамматики увеличивается, то пропорционально увеливается и количество разборов. Не редко NLTK дают ложные разборы (потому что относительно точную грамматику составить, конечно, можно, но сложно)

Давайте возьмём иконичный (дебильный) пример из NLTK.

 Преложение fish fish fish fish fis, которое, кстати говоря, является абсолютно грамматическим английским предложением неоднозначно в нашей грамматике. Как автоматически понять, какой разбор лучше?

In [75]:
>>> grammar = nltk.CFG.fromstring("""
... S -> NP V NP
... NP -> NP Sbar
... Sbar -> NP V
... NP -> 'fish'
... V -> 'fish'
... """)

# 'fish that other fish fish are in the habit of fishing fish themselves'.
>>> tokens = ["fish"] * 5
>>> cp = nltk.ChartParser(grammar)
>>> for tree in cp.parse(tokens):
...     print(tree)
# ну и какое верное, кстати говоря?

(S (NP fish) (V fish) (NP (NP fish) (Sbar (NP fish) (V fish))))
(S (NP (NP fish) (Sbar (NP fish) (V fish))) (V fish) (NP fish))


какие есть решения?

1. Особые классы глаголов

In [76]:
rule_grammar = nltk.CFG.fromstring("""
VPpred -> Vpred Adj
VPn -> Vn NP	
VPs -> Vs S	
VPphrase -> Vphrase NP PP	
Vpred -> 'was'
Vn -> 'saw'
Vs -> 'thought'
Vphrase -> 'put'

 """)


2. Weighted grammar

У нас есть большой корпус деревьев в NLTK. Можно посчитать вероятность каждой конструкции. Чуть выше мы видели код, который найдет все Vp -> V S. Можно такие вычисления проделать для всего и добавить это в нашу грамматику в таком формате.

In [77]:
	
grammar = nltk.PCFG.fromstring("""
    S    -> NP VP              [1.0]
    VP   -> TV NP              [0.4]
    VP   -> IV                 [0.3]
    VP   -> DatV NP NP         [0.3]
    TV   -> 'saw'              [1.0]
    IV   -> 'ate'              [1.0]
    DatV -> 'gave'             [1.0]
    NP   -> 'telescopes'       [0.8]
    NP   -> 'Jack'             [0.2]
    """)

In [79]:
>>> viterbi_parser = nltk.ViterbiParser(grammar) #снова новый парсер
>>> for tree in viterbi_parser.parse(['Jack', 'saw', 'telescopes']):
...     print(tree)

(S (NP Jack) (VP (TV saw) (NP telescopes))) (p=0.064)


Задание 1.

Любым доступным способом сделайте дерево для  Buffalo buffalo Buffalo buffalo buffalo buffalo Buffalo buffalo. Посмотрите на дерево по ссылке и напишите грамматику, чтобы получилось также и был только один вариант  http://en.wikipedia.org/wiki/Buffalo_buffalo_Buffalo_buffalo_buffalo_buffalo_Buffalo_buffalo. 


In [ ]:
# Your code

3. Морфология

In [80]:
VP[TENSE=?t, NUM=?n] -> V[SUBCAT=intrans, TENSE=?t, NUM=?n]
VP[TENSE=?t, NUM=?n] -> V[SUBCAT=trans, TENSE=?t, NUM=?n] NP
VP[TENSE=?t, NUM=?n] -> V[SUBCAT=clause, TENSE=?t, NUM=?n] SBar

V[SUBCAT=intrans, TENSE=pres, NUM=sg] -> 'disappears' | 'walks'
V[SUBCAT=trans, TENSE=pres, NUM=sg] -> 'sees' | 'likes'
V[SUBCAT=clause, TENSE=pres, NUM=sg] -> 'says' | 'claims'

V[SUBCAT=intrans, TENSE=pres, NUM=pl] -> 'disappear' | 'walk'
V[SUBCAT=trans, TENSE=pres, NUM=pl] -> 'see' | 'like'
V[SUBCAT=clause, TENSE=pres, NUM=pl] -> 'say' | 'claim'

V[SUBCAT=intrans, TENSE=past, NUM=?n] -> 'disappeared' | 'walked'
V[SUBCAT=trans, TENSE=past, NUM=?n] -> 'saw' | 'liked'
V[SUBCAT=clause, TENSE=past, NUM=?n] -> 'said' | 'claimed'

SyntaxError: ignored

4. Всё вместе

## Разумеется, всё у нас уже готово в NLTK

In [83]:
nltk.download ('book_grammars')

[nltk_data] Downloading package book_grammars to /root/nltk_data...
[nltk_data]   Unzipping grammars/book_grammars.zip.


True

In [84]:
nltk.data.show_cfg('grammars/book_grammars/feat1.fcfg')

% start S
# ###################
# Grammar Productions
# ###################
S[-INV] -> NP VP
S[-INV]/?x -> NP VP/?x
S[-INV] -> NP S/NP
S[-INV] -> Adv[+NEG] S[+INV]
S[+INV] -> V[+AUX] NP VP
S[+INV]/?x -> V[+AUX] NP VP/?x
SBar -> Comp S[-INV]
SBar/?x -> Comp S[-INV]/?x
VP -> V[SUBCAT=intrans, -AUX]
VP -> V[SUBCAT=trans, -AUX] NP
VP/?x -> V[SUBCAT=trans, -AUX] NP/?x
VP -> V[SUBCAT=clause, -AUX] SBar
VP/?x -> V[SUBCAT=clause, -AUX] SBar/?x
VP -> V[+AUX] VP
VP/?x -> V[+AUX] VP/?x
# ###################
# Lexical Productions
# ###################
V[SUBCAT=intrans, -AUX] -> 'walk' | 'sing'
V[SUBCAT=trans, -AUX] -> 'see' | 'like'
V[SUBCAT=clause, -AUX] -> 'say' | 'claim'
V[+AUX] -> 'do' | 'can'
NP[-WH] -> 'you' | 'cats'
NP[+WH] -> 'who'
Adv[+NEG] -> 'rarely' | 'never'
NP/NP ->
Comp -> 'that'


In [85]:
>>> tokens = 'who do you claim that you like'.split()
>>> from nltk import load_parser
>>> cp = load_parser('grammars/book_grammars/feat1.fcfg')
>>> for tree in cp.parse(tokens):
...     print(tree)

(S[-INV]
  (NP[+WH] who)
  (S[+INV]/NP[]
    (V[+AUX] do)
    (NP[-WH] you)
    (VP[]/NP[]
      (V[-AUX, SUBCAT='clause'] claim)
      (SBar[]/NP[]
        (Comp[] that)
        (S[-INV]/NP[]
          (NP[-WH] you)
          (VP[]/NP[] (V[-AUX, SUBCAT='trans'] like) (NP[]/NP[] )))))))
